## Analyze IMU Errors on the dataset


In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import json
import matplotlib.pyplot as plt

# Get environment variables
import sys

sys.path.append("..")  # Add the parent directory of notebooks to sys.path

from lac.plotting import pose_traces

%load_ext autoreload
%autoreload 2

# IMU reverse engineering

Reference: `Leaderboard/leaderboard/agents/imu.py`

- `ang_vel = np.array([angvel_mat[2, 1], angvel_mat[0, 2], angvel_mat[1, 0]])`


In [17]:
from scipy.spatial.transform import Rotation

from lac.util import skew_symmetric, normalize_rotation_matrix

In [18]:
# casename = "20hz_seed4"
# data_path = "../../output/imu/" + casename + ".json"
data_path = "../../output/imu_20hz/data_log.json"
json_data = json.load(open(f"{data_path}"))

poses = []
imu_data = []

for frame in json_data["frames"]:
    poses.append(np.array(frame["pose"]))
    imu_data.append(np.array(frame["imu"]))

imu_data = np.array(imu_data)

## Backing out rotations from gyro


In [ ]:
i = 1000

R_0 = poses[i - 1][:3, :3]

w_hat = skew_symmetric(imu_data[i, 3:])
dt = 0.05
R_1 = (np.eye(3) - w_hat * dt).T @ R_0
R_1 = normalize_rotation_matrix(R_1)
print("estimated: \n", R_1)
print("actual: \n", poses[i][:3, :3])
print("diff: \n", R_1 - poses[i][:3, :3])

In [31]:
import symforce

try:
    symforce.set_epsilon_to_symbol()
except symforce.AlreadyUsedEpsilon:
    print("Already set symforce epsilon")
    pass
import symforce.symbolic as sf
from symforce.notebook_util import display

In [ ]:
R0 = sf.Rot3.from_rotation_matrix(poses[i - 1][:3, :3])
R1 = sf.Rot3.symbolic("R1")
A = sf.M33.eye() - (R0 * R1.inverse()).to_rotation_matrix()
A[0, 0]

In [62]:
from symforce.values import Values
from symforce.opt.factor import Factor

i = 1000

values = Values(
    R_prev=sf.Rot3.from_rotation_matrix(poses[i - 1][:3, :3]),
    R_curr=sf.Rot3.from_rotation_matrix(poses[i - 1][:3, :3]),
    omega=sf.V3(imu_data[i, 3:]),
    dt=0.05,
)


def residual(R_prev: sf.Rot3, R_curr: sf.Rot3, omega: sf.V3, dt: float) -> sf.V3:
    Omega = (sf.M33.eye() - (R_prev * R_curr.inverse()).to_rotation_matrix()) / dt
    return sf.V3(omega[0] - Omega[2, 1], omega[1] - Omega[0, 2], omega[2] - Omega[1, 0])


factors = [Factor(residual=residual, keys=["R_prev", "R_curr", "omega", "dt"])]

In [ ]:
from symforce.opt.optimizer import Optimizer

optimizer = Optimizer(
    factors=factors,
    optimized_keys=["R_curr"],
    debug_stats=True,
)
result = optimizer.optimize(values)

In [ ]:
result.optimized_values["R_curr"].to_rotation_matrix() - poses[i][:3, :3]

In [ ]:
from scipy.optimize import fsolve, root

i = 1
R_0 = poses[i - 1][:3, :3]
omega = imu_data[i, 3:]
dt = 0.05


def equations(vars):
    qr, qi, qj, qk = vars
    return [
        (
            -2 * (qi * qk - qj * qr) * R_0[0, 0]
            - 2 * (qj * qk + qi * qr) * R_0[0, 1]
            - (1 - 2 * (qi**2 + qj**2)) * R_0[0, 2]
        )
        / dt
        - omega[2],
        (
            -((1 - 2 * (qj**2 + qk**2)) * R_0[1, 0])
            - 2 * (qi * qj - qk * qr) * R_0[1, 1]
            - 2 * (qi * qk + qj * qr) * R_0[1, 2]
        )
        / dt
        - omega[0],
        (
            -2 * (qi * qj + qk * qr) * R_0[2, 0]
            - (1 - 2 * (qi**2 + qk**2)) * R_0[2, 1]
            - 2 * (qj * qk - qi * qr) * R_0[2, 2]
        )
        / dt
        - omega[1],
        qr**2 + qi**2 + qj**2 + qk**2 - 1,
    ]


initial_guess = Rotation.from_matrix(R_0).as_quat(scalar_first=True)
qr, qi, qj, qk = fsolve(equations, initial_guess)
R_1 = Rotation.from_quat([qi, qj, qk, qr]).as_matrix()
print("estimated: \n", R_1)
print("actual: \n", poses[i][:3, :3])
print("error: \n", R_1 - poses[i][:3, :3])

In [ ]:
solution = root(equations, initial_guess, method="hybr", tol=1e-12)

R_1 = Rotation.from_quat(solution.x).as_matrix()
print("estimated: \n", R_1)
print("actual: \n", poses[i][:3, :3])
print("error: \n", R_1 - poses[i][:3, :3])

In [ ]:
from mpmath import mp, findroot

mp.dps = 50


# Define your function to take separate arguments
def equations(qr, qi, qj, qk):
    return [
        (
            -2 * (qi * qk - qj * qr) * R_0[0, 0]
            - 2 * (qj * qk + qi * qr) * R_0[0, 1]
            - (1 - 2 * (qi**2 + qj**2)) * R_0[0, 2]
        )
        / dt
        - omega[2],
        (
            -((1 - 2 * (qj**2 + qk**2)) * R_0[1, 0])
            - 2 * (qi * qj - qk * qr) * R_0[1, 1]
            - 2 * (qi * qk + qj * qr) * R_0[1, 2]
        )
        / dt
        - omega[0],
        (
            -2 * (qi * qj + qk * qr) * R_0[2, 0]
            - (1 - 2 * (qi**2 + qk**2)) * R_0[2, 1]
            - 2 * (qj * qk - qi * qr) * R_0[2, 2]
        )
        / dt
        - omega[1],
        qr**2 + qi**2 + qj**2 + qk**2 - 1,  # Quaternion normalization constraint
    ]


initial_guess = Rotation.from_matrix(poses[i][:3, :3]).as_quat(scalar_first=True)
initial_guess_mpf = [mp.mpf(val) for val in initial_guess]
solution = findroot(equations, initial_guess_mpf, tol=1e-12)
solution = [float(s) for s in solution]
print("solution: ", solution)
R_1 = Rotation.from_quat(solution).as_matrix()
print("estimated: \n", R_1)
print("actual: \n", poses[i][:3, :3])
print("error: \n", R_1 - poses[i][:3, :3])

In [ ]:
qr, qi, qj, qk = solution
equations(qr, qi, qj, qk)

In [ ]:
((poses[i][:3, :3] - R_0) / dt) @ poses[i][:3, :3].T

In [ ]:
((R_1 - R_0) / dt) @ R_1.T

In [ ]:
from sympy import symbols, Eq, nsolve
from scipy.spatial.transform import Rotation
import numpy as np
from sympy.mpmath import mp

# Define symbolic variables
qr, qi, qj, qk = symbols("qr qi qj qk")

# Given data
i = 1
R_0 = poses[i - 1][:3, :3]  # Previous rotation matrix
omega = imu_data[i, 3:]  # Angular velocity
dt = 0.05  # Time step

# Define quaternion rotation matrix elements
r11 = 1 - 2 * (qj**2 + qk**2)
r12 = 2 * (qi * qj - qk * qr)
r13 = 2 * (qi * qk + qj * qr)
r21 = 2 * (qi * qj + qk * qr)
r22 = 1 - 2 * (qi**2 + qk**2)
r23 = 2 * (qj * qk - qi * qr)
r31 = 2 * (qi * qk - qj * qr)
r32 = 2 * (qj * qk + qi * qr)
r33 = 1 - 2 * (qi**2 + qj**2)

# Define the equations
eq1 = Eq((-r31 * R_0[0, 0] - r32 * R_0[0, 1] - r33 * R_0[0, 2]) / dt - omega[2], 0)
eq2 = Eq((-r11 * R_0[1, 0] - r12 * R_0[1, 1] - r13 * R_0[1, 2]) / dt - omega[0], 0)
eq3 = Eq((-r21 * R_0[2, 0] - r22 * R_0[2, 1] - r23 * R_0[2, 2]) / dt - omega[1], 0)
eq4 = Eq(qr**2 + qi**2 + qj**2 + qk**2 - 1, 0)  # Unit quaternion constraint

# Initial guess using Scipy
initial_guess = Rotation.from_matrix(poses[i][:3, :3]).as_quat(scalar_first=True)

# Solve using nsolve
solution = nsolve((eq1, eq2, eq3, eq4), (qr, qi, qj, qk), initial_guess)

# Extract quaternion solution
qr, qi, qj, qk = solution

print("Solved quaternion:", qr, qi, qj, qk)
R_1 = Rotation.from_quat([qi, qj, qk, qr]).as_matrix()
print("estimated: \n", R_1)
print("actual: \n", poses[i][:3, :3])
print("error: \n", R_1 - poses[i][:3, :3])

In [ ]:
poses[i - 1][:3, :3]

In [ ]:
imu_data[i, 3:]

In [ ]:
(np.eye(3) - skew_symmetric(omega) * dt).T @ R_0

In [ ]:
Omega = ((poses[i][:3, :3] - poses[i - 1][:3, :3]) / dt) @ poses[i][:3, :3].T
print(Omega)
print(Omega[2, 1], Omega[0, 2], Omega[1, 0])
print(imu_data[i, 3:])

In [ ]:
imu_data[i, 3]

In [23]:
R_0 = poses[0][:3, :3]

imu_rotations = []
imu_rotations.append(R_0)

for i in range(1, len(poses)):
    w_hat = skew_symmetric(imu_data[i, 3:])
    dt = json_data["frames"][i]["mission_time"] - json_data["frames"][i - 1]["mission_time"]
    R_1 = (np.eye(3) - w_hat * dt).T @ R_0
    R_1 = normalize_rotation_matrix(R_1)
    imu_rotations.append(R_1)
    R_0 = R_1

In [ ]:
gt_rotations_euler = np.zeros((len(poses), 3))
imu_rotations_euler = np.zeros((len(poses), 3))

for i in range(len(poses)):
    gt_rotations_euler[i] = Rotation.from_matrix(poses[i][:3, :3]).as_euler("xyz", degrees=True)
    imu_rotations_euler[i] = Rotation.from_matrix(imu_rotations[i]).as_euler("xyz", degrees=True)

fig, axes = plt.subplots(3, 1, figsize=(12, 8))
pos_labels = ["Roll (deg)", "Pitch (deg)", "Yaw (deg)"]
for i in range(3):
    ax = axes[i]
    ax.plot(gt_rotations_euler[:, i], label="True")
    ax.plot(imu_rotations_euler[:, i], label="IMU", alpha=0.5)
    ax.legend()
    ax.grid()
    ax.set_xlabel("Timestep (0.1 s)")
    ax.set_ylabel(f"{pos_labels[i]}")
plt.subplots_adjust(wspace=0.0, hspace=0.3)

In [ ]:
# Error plots
fig, axes = plt.subplots(3, 1, figsize=(12, 8))
pos_labels = ["Roll", "Pitch", "Yaw"]
for i in range(3):
    ax = axes[i]
    ax.plot(gt_rotations_euler[:, i] - imu_rotations_euler[:, i])
    ax.grid()
    ax.set_xlabel("Timestep (0.1 s)")
    ax.set_ylabel(f"{pos_labels[i]} Error (deg)")
plt.subplots_adjust(wspace=0.0, hspace=0.3)

In [ ]:
print("Average error in degrees")
print(np.mean(np.abs(gt_rotations_euler - imu_rotations_euler), axis=0))

## Translations from accelerometer


In [31]:
gravity = np.array([0.0, 0.0, 1.6220])  # m/s^2

use_true_rot = False

t_0 = poses[0][:3, 3]
t_1 = poses[1][:3, 3]
v_1 = (t_1 - t_0) / (
    json_data["frames"][1]["mission_time"] - json_data["frames"][0]["mission_time"]
)

imu_translations = []
imu_translations.append(t_0)
imu_translations.append(t_1)

for i in range(2, len(poses)):
    dt = json_data["frames"][i]["mission_time"] - json_data["frames"][i - 1]["mission_time"]
    a = imu_data[i, :3]

    if use_true_rot:
        R_i = poses[i][:3, :3]
    else:
        R_i = imu_rotations[i]

    v_2 = v_1 + (R_i @ a - gravity) * dt
    t_2 = t_1 + v_1 * dt

    imu_translations.append(t_2)
    t_0 = t_1
    t_1 = t_2
    v_1 = v_2

imu_translations = np.array(imu_translations)

gt_translations = np.zeros((len(poses), 3))
for i in range(len(poses)):
    gt_translations[i] = poses[i][:3, 3]

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(12, 8))
pos_labels = ["X", "Y", "Z"]

N = -1

for i in range(3):
    ax = axes[i]
    ax.plot(gt_translations[:N, i], label="True")
    ax.plot(imu_translations[:N, i], label="IMU", alpha=0.5)
    ax.legend()
    ax.grid()
    ax.set_xlabel("Timestep (0.1 s)")
    ax.set_ylabel(f"{pos_labels[i]}")
plt.subplots_adjust(wspace=0.0, hspace=0.3)

In [ ]:
gt_translations - imu_translations